In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("bot_detection_data.csv")

In [3]:
df.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   User ID         50000 non-null  int64 
 1   Username        50000 non-null  object
 2   Tweet           50000 non-null  object
 3   Retweet Count   50000 non-null  int64 
 4   Mention Count   50000 non-null  int64 
 5   Follower Count  50000 non-null  int64 
 6   Verified        50000 non-null  bool  
 7   Bot Label       50000 non-null  int64 
 8   Location        50000 non-null  object
 9   Created At      50000 non-null  object
 10  Hashtags        41659 non-null  object
dtypes: bool(1), int64(5), object(5)
memory usage: 3.9+ MB


In [5]:
df.drop(columns=["Username","Created At","Hashtags","Location","Tweet","User ID"],inplace=True)

In [6]:
df.head()

,Retweet Count,Mention Count,Follower Count,Verified,Bot Label
0,85,1,2353,False,1
1,55,5,9617,True,0
2,6,2,4363,True,0
3,54,5,2242,True,1
4,26,3,8438,False,1


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=["Bot Label"]),
                                              df["Bot Label"],
                                              test_size=0.2,
                                              random_state=42)

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [9]:
df.head()

,Retweet Count,Mention Count,Follower Count,Verified,Bot Label
0,85,1,2353,False,1
1,55,5,9617,True,0
2,6,2,4363,True,0
3,54,5,2242,True,1
4,26,3,8438,False,1


In [10]:
trf1= ColumnTransformer([
    ("ohe_Verified",OneHotEncoder(sparse= False,handle_unknown = "ignore"),[3])
],remainder="passthrough")

In [11]:
trf2 = ColumnTransformer([
    ("scale", MinMaxScaler(),slice(0,5))
])

In [24]:
trf3 = SelectKBest(score_func=chi2,k=5)

In [25]:
trf4= DecisionTreeRegressor()

In [26]:
pipe = Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4)
])

In [27]:
pipe.fit(x_train,y_train)

C:\Users\shrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_Verified',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [3])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 5, None))])),
                ('trf3',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000001FA3BD42D40>)),
                ('trf4', DecisionTreeRegressor())])

In [28]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_Verified',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [3])]),
 'trf2': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 5, None))]),
 'trf3': SelectKBest(k=5, score_func=<function chi2 at 0x000001FA3BD42D40>),
 'trf4': DecisionTreeRegressor()}

In [29]:
y_pred= pipe.predict(x_test).round()

In [30]:
y_pred

array([1., 1., 0., ..., 1., 1., 1.])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5028

In [32]:
import pickle
pickle.dump(pipe,open("pipe.pkl","wb"))

In [33]:
#launching the code
pipe= pickle.load(open("pipe.pkl","rb"))

In [39]:
#assuming user input
test_input = np.array([5,5,2523,"True"],dtype= object).reshape(1,4)

In [40]:
pipe.predict(test_input)

C:\Users\shrey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([1.])